In [15]:
from boto3.dynamodb.conditions import Attr
res = (Attr('test').eq(1) & Attr('a').lt(2)) | Attr('b').contains('a')()}

In [95]:
test = {'a': 1}
{**test, 'b': 2, 'a':2}

{'a': 2, 'b': 2}

In [70]:
test = "( ( A__eq:int:1)AND(B__lt:date:2))OR(C__gte:str:3)"

In [75]:
# STEP 1
# Split string into a list of untyped tokens
import re
separators = ['\(', '\)', ':']

# re.partition(re.split(''))
rstr = "({})".format('|'.join(separators))
splitted = re.split(re.compile(rstr), test)
tokens_to_ignore = {':', ''}
no_type_tokens = list(filter(lambda tok: tok not in tokens_to_ignore, [s.strip() for s in splitted]))
no_type_tokens

['(',
 '(',
 'A__eq',
 'int',
 '1',
 ')',
 'AND',
 '(',
 'B__lt',
 'date',
 '2',
 ')',
 ')',
 'OR',
 '(',
 'C__gte',
 'str',
 '3',
 ')']

In [95]:
# Get token primary and secondary types


class Token:
    def __init__(self, literal, primary_type, secondary_type):
        if primary_type not in registered_token_types or secondary_type not in registered_token_types[primary_type]:
            raise Exception('Attempted to tokenize an unregistered primary/secondary type')
        
        self.literal = literal
        self.primary_type = primary_type
        self.secondary_type = secondary_type
        self.string_notation = f'{primary_type}:{secondary_type}'
    
    def set_string_notation(self):
        self.string_notation = f'{primary_type}:{secondary_type}'
    
    def update_secondary_type(self, new_secondary_type):
        self.secondary_type = new_secondary_type
        self.set_string_notation()
    
    def __str__(self):
        return self.string_notation


class ContextToken(Token):
    pass


import operator
token_operator = {'and': operator.and_, 'or': operator.or_}
token_precedence = {'and': 2, 'or': 1}

class LogicalToken(Token):
    def __init__(self, literal, primary_type, secondary_type):
        super().__init__(literal, primary_type, secondary_type)
        self.precedence = token_precedence[self.secondary_type]
        self.operator = token_operator[self.secondary_type]

from decimal import Decimal
from datetime import datetime, date
value_types = {
    'int': int,
    'str': str,
    'float': float,
    'decimal': Decimal,
    'date': date,
    'datetime': datetime
}
    
class ExpressionToken(Token):
    def get_field_and_lookup_expression(self):
        if not self.secondary_type == 'field':
            return None
        
        splitted_field = self.literal.split('__')
        splitted_count = len(splitted_field)
        if splitted_count == 1:
            lookup_expression = 'eq'
        elif splitted_count in [2, 3]:
            lookup_expression = splitted_field[-1]
        else:
            raise Exception('Invalid lookup expression!')
        
        return splitted_field[0], lookup_expression
    
    def get_value_type_casting_method(self):
        if not self.secondary_type == 'value_type':
            return None
        
        try:
            return value_types[self.literal]
        except KeyError:
            raise Exception('Invalid value type!')
        


registered_token_types = {
    'context': {'open', 'close'},
    'logical': {'and', 'or'},
    'expression': {'field', 'value_type', 'value'}
}

general_token_types = {
    '(': ('context', 'open'),
    ')': ('context', 'close'),
    'AND': ('logical', 'and'),
    'OR': ('logical', 'or')
}

field_lambda = lambda _: ('expression', 'field')
expression_token_types = {
    # previous token: (typ, subtype)
    None: field_lambda,
    'context:open': field_lambda,
    'expression:field': lambda next_literal: ('expression', 'value') if next_literal in [None, ')'] else ('expression', 'value_type'),
    'expression:value_type': lambda _: ('expression', 'value')
}

primary_type_to_Token = {
    'context': ContextToken,
    'logical': LogicalToken,
    'expression': ExpressionToken
}

def get_instanced_token(current_literal: str, previous_token_string_notation: str, next_literal: str):
    try:
        primary_type, secondary_type = general_token_types[current_literal]
    except KeyError:
        try:
            primary_type, secondary_type = expression_token_types[previous_token_string_notation](next_literal)
        except KeyError:
            raise Exception(f'Couldn\'t determine token type!\nCurrent -> {current_literal}\nPrevious -> {previous_token_string_notation}\nNext -> {next_literal}')
    
    if primary_type == None or secondary_type == None:
        return None
    
    return primary_type_to_Token[primary_type](current_literal, primary_type, secondary_type)
        
        


In [74]:
instanced_tokens = []

previous_token = None
for index, current_token in enumerate(no_type_tokens):

    try:
        next_token = no_type_tokens[index + 1]
    except IndexError:
        next_token = None

    instanced_token = get_instanced_token(current_token, previous_token, next_token)
    instanced_tokens.append(instanced_token)
    previous_token = str(instanced_token)

print(test)
[f'{str(t)} -> {t.literal}' for t in instanced_tokens]

( ( A__eq:int:1)AND(B__lt:date:2))OR(C__gte:str:3)


['context:open -> (',
 'context:open -> (',
 'expression:field -> A__eq',
 'expression:value_type -> int',
 'expression:value -> 1',
 'context:close -> )',
 'logical:and -> AND',
 'context:open -> (',
 'expression:field -> B__lt',
 'expression:value_type -> date',
 'expression:value -> 2',
 'context:close -> )',
 'context:close -> )',
 'logical:or -> OR',
 'context:open -> (',
 'expression:field -> C__gte',
 'expression:value_type -> str',
 'expression:value -> 3',
 'context:close -> )']

In [96]:
class Node:
    def __init__(self, left_child=None, right_child=None):
        self.left_child = left_child
        self.right_child = right_child

class ExpressionNode(Node):
    def __init__(self, *args):
        
        if len(args) not in [2, 3]:
            raise Exception(f'Couldn\'t build ExpressionNode! Tokens -> {args}')
        
        field_token = args[0]
        value_token = args[-1]
        value_type_token = args[1] if len(args) == 3 else None
            
        if any(not isinstance(tok, ExpressionToken) for tok in [field_token, value_token]):
            raise Exception('Cannot build expression node out of non ExpressionTokens!')
        
        field, lookup_expression = field_token.get_lookup_expression()
        self.field = field
        self.lookup_expression = lookup_expression
        
        value_type = None if not value_type_token else value_type_token.get_value_type_casting_method()
        self.value = value_type(value_token.literal) if value_type else value_token.literal
        
class LogicalNode(Node):
    def __init__(self, logical_token, left_child, right_child):
        self.operator = logical_token.operator
        super().__init__(left_child, right_child)

        
        

In [93]:

next_expected_tokens = {
    None: {'context:open', 'expression:field'},
    'context:open': {'context:open', 'expression:field'},
    'context:close': {'context:close', 'logical:and', 'logical:or'},
    'logical:and': {'context:open'},
    'logical:or': {'context:open'},
    'expression:field': {'expression:value_type', 'expression:value'},
    'expression:value_type': {'expression:value'},
    'expression:value': {'context:close'}
}



# build AST
from collections import deque
from deepcopy import deepcopy

contexts_stack = []
current_context_head = None
current_expression_tokens = []
parent_node = None
expected_tokens = next_expected_tokens[None]

instanced_tokens.append(None)  # End
for tok in instanced_tokens:
    tok_repr = str(tok)
    if tok_repr not in expected_tokens:
        raise Exception(f'Unexpected token encountered -> {tok_repr}, one of the following was expected -> {expected_tokens}')
    
    if isinstance(tok, ContextToken) and tok.secondary_type == 'open':
        if tok.secondary_type == 'open':
            contexts_stack.append(None)
        elif tok.secondary_type == 'close':
            if not contexts_stack:
                raise Exception('Attempted to close unexistent context!')

            if current_expression_tokens:
                try:
                    expression_node = ExpressionNode(*current_expression_tokens)
                except Exception as e:
                    raise Exception(f'Couldn\'t parse expression node! -> {e}')
                current_expression_tokens = []

                if contexts_stack[-1] == None:
                    current_context_head = deepcopy(expression_node)
                elif isinstance(contexts_stack[-1], LogicalNode):
                    current_context_head.right_child = deepcopy(expression_node)
                else:
                    raise Exception('Found an unexpected expression??')
                expression_node = None
            else:
                

    elif isinstance(tok, ExpressionToken):
        current_expression_tokens.append(tok)            
            
            
        
    
    expected_tokens = next_expected_tokens[str(tok)]


TypeError: op_and_ expected 2 arguments, got 3

In [100]:
str(None)

'None'